# Image compression met PCA

In dit bestand wordt getoond hoe je met behulp van PCA een beeld kan comprimeren.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd

Lees een beeld:

In [ ]:
img = np.array(Image.open('Flower.jpg'))
print(img.shape)

Het beeld heeft een resolutie van 2448 op 3264 en bevat 3 waarden per pixel (red, green, blue).

Toon het beeld:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img)
# fig.show()

Splits beeld op in rood, groen en blauw

In [ ]:
red = img[:,:,0]
green = img[:,:,1]
blue = img[:,:,2]
print(red.shape)

Transformeer iedere kleur naar een 42-dimensionale ruimte (ipv 3264):

In [ ]:
pca_dim = 42

pca_red_model = PCA(n_components= pca_dim)
pca_red = pca_red_model.fit_transform(red)

pca_green_model = PCA(n_components= pca_dim)
pca_green = pca_green_model.fit_transform(green)

pca_blue_model = PCA(n_components= pca_dim)
pca_blue = pca_blue_model.fit_transform(blue)

print(pca_red.shape)

Hebben we nu de rijen of de kolommen gecomprimeerd?

Reconstrueer het beeld aan de hand van de gecomprimeerde versie:

In [ ]:
pc_red_inverted = pca_red_model.inverse_transform(pca_red)
pc_green_inverted = pca_green_model.inverse_transform(pca_green)
pc_blue_inverted = pca_blue_model.inverse_transform(pca_blue)
img_compressed = (np.dstack((pc_red_inverted, pc_green_inverted, pc_blue_inverted))).astype(np.uint8)
print(img_compressed.shape)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img_compressed)
# fig.show()

Hoe goed was de compressie?

In [ ]:
print(pca_red_model.explained_variance_ratio_.sum())
print(pca_green_model.explained_variance_ratio_.sum())
print(pca_blue_model.explained_variance_ratio_.sum())

Hoe groot is de compression ratio?

We berekenen hoeveel bytes er nodig waren voor het originele beeld:
1 byte per kleur (red, green, blue)
dus 3 bytes per pixel

In [ ]:
original_size = pd.Series(img.shape).prod()
print(original_size)

Nu berekenen we hoeveel bytes er nodig zijn voor de gecomprimeerde versie.
Er zijn nu slechts 42 kolommen, maar we hebben floating point waarden nodig om de waarden te stockeren.
We moeten de transformatiematrix ook stockeren.  Deze bevat pca_dim x het aantal oorspronkelijke kolommen floating point waarden.

In [ ]:
number_of_colours = 3
bytes_in_float = 4
compressed_size = pd.Series(pca_red.shape).prod() * number_of_colours * bytes_in_float
compressed_size = compressed_size + pca_dim * img.shape[1] * bytes_in_float
print(compressed_size)

Hoeveel procent is de gecomprimeerde versie van de originele?

In [ ]:
print(compressed_size / original_size)

Wat is dus de compression ratio?

In [ ]:
print(original_size / compressed_size)